# First model attempt

In [1]:
import sys
sys.path.insert(1, '/Users/lauradellantonio/neuefische/Capstone/capstone')

In [2]:
import pandas as pd
import numpy as np

import functions.functions_data
import functions.functions_mask_wearing
import functions.functions_corona_stats
import functions.functions_HDI
import functions.functions_mask_req

pd.set_option('display.max_columns', None) # To display all columns

import warnings
warnings.filterwarnings(action='ignore')

## Reading in and adjusting data

In [3]:
# Reading in survey data and joining it into one dataframe
dfs_country = functions.functions_data.get_data("/Users/lauradellantonio/neuefische/Capstone/capstone/data/CMU_Global_data/Full_Survey_Data/country/smooth/", "country")
countries = pd.concat(dfs_country, ignore_index=True)

Read in data completed.


In [4]:
# Adding the real COVID numbers to the dataframe
covid_cases = pd.read_csv("/Users/lauradellantonio/neuefische/Capstone/capstone/data/Corona_stats/owid-covid-data.csv")
countries = functions.functions_corona_stats.merge_corona_stats(countries,covid_cases)

Merging corona stats completed.


In [5]:
# Sorting out the date column within the dataframe
countries = functions.functions_data.insert_month(countries)

Created month column.


In [6]:
# Dealing with the NaN in the mask wearing column
countries = functions.functions_mask_wearing.deal_with_NaNs_masks(countries)

NaNs before update: 163383
NaNs after update: 0
Updated.


In [7]:
# Getting the dictionaries for the hdi and the hdi-levels.
dict_hdi, dict_hdi_levels = functions.functions_HDI.get_hdi("/Users/lauradellantonio/neuefische/Capstone/capstone/data","hdro_statistical_data_tables_1_15_d1_d5.xlsx")

# Creating a new column with the hdi and the hdi-levels in the regions dataframe
countries = functions.functions_HDI.create_hdi_columns(countries, dict_hdi, dict_hdi_levels)

Creating dictionaries for hdi and hdi-levels completed.
Creating hdi list completed.
Creating hdi-level list completed.


In [10]:
# Adding the mask wearing requirements to the dataframe
mask_wearing_requirements = pd.read_csv("/Users/lauradellantonio/neuefische/Capstone/capstone/data/data-nbhtq.csv")
countries = functions.functions_mask_req.merge_mask_requirements(countries,mask_wearing_requirements)
countries = functions.functions_mask_req.add_requirement_by_date(countries)

Merging mask wearing requirements completed.
Feature engineering completed.


## Selecting relevant columns

In [11]:
df = countries.copy()

In [16]:
# Date used as index

In [19]:
df = df[df["age_bucket"]=="overall"]
df = df[df["gender"]=="overall"]

In [52]:
date = ["date"]

columns_general = ["iso_code", "hdi", "median_age", "aged_65_older", "total_cases_per_million", "new_cases_smoothed_per_million",
                   "total_deaths_per_million", "new_deaths_smoothed_per_million"]

columns_social_distancing = ["smoothed_pct_worked_outside_home_weighted", "smoothed_pct_grocery_outside_home_weighted", "smoothed_pct_ate_outside_home_weighted", 
                             "smoothed_pct_spent_time_with_non_hh_weighted", "smoothed_pct_attended_public_event_weighted", "smoothed_pct_used_public_transit_weighted", 
                             "smoothed_pct_direct_contact_with_non_hh_weighted", "smoothed_pct_no_public_weighted"]

columns_mask_wearing = ["smoothed_pct_wear_mask_all_time_weighted", "smoothed_pct_wear_mask_most_time_weighted", "smoothed_pct_wear_mask_half_time_weighted", 
                        "smoothed_pct_wear_mask_some_time_weighted","smoothed_pct_wear_mask_none_time_weighted"]

columns_mask_req = ["cur_mask_recommended", "cur_mask_required_full_country", "cur_mask_not_required", "cur_mask_not_required_recommended", "cur_mask_not_required_universal", 
                    "cur_mask_required_part_country", "cur_mask_everywhere_in_public", "cur_mask_public_indoors", "cur_mask_public_transport"]

columns_interest = date + columns_general + columns_social_distancing + columns_mask_wearing + columns_mask_req

In [45]:
df_select = df[columns_interest]

In [51]:
df_select.set_index("date", inplace=True)

## VARMAX models attempt
https://www.statsmodels.org/stable/examples/notebooks/generated/statespace_varmax.html

In [12]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# If we predict new/total cases we're not allowed to have any of the deaths/cases columns in our train dataset 

In [ ]:
# Plot
ax = res.impulse_responses(10, orthogonalized=True).plot(figsize=(13,3))
ax.set(xlabel='t', title='Responses to a shock to `dln_inv`');

In [ ]:
mod = sm.tsa.VARMAX(endog[['dln_inv', 'dln_inc']], order=(0,2), error_cov_type='diagonal')
res = mod.fit(maxiter=1000, disp=False)
print(res.summary())

In [ ]:
mod = sm.tsa.VARMAX(endog[['dln_inv', 'dln_inc']], order=(1,1))
res = mod.fit(maxiter=1000, disp=False)
print(res.summary())